In [ ]:
using Pkg
Pkg.activate("./Project.toml");

In [ ]:
using CSV,DataFrames,Plots,Measures,ColorSchemes,GaussianMixtures,Distributions,StaticArrays,Random; gr();
Plots.gr_cbar_width[] = 0.015
plt = palette(:jet);

In [ ]:
pwd()

## Load the force versus extension data

In [ ]:
flpath = "extensionfiles/";

In [ ]:
df0 = CSV.read(flpath*"mmcdh23EC19-20_0CA_mstext.csv",DataFrame)
df1 = CSV.read(flpath*"mmcdh23EC19-20_1CA_mstext.csv",DataFrame)
df2 = CSV.read(flpath*"mmcdh23EC19-20_2CA_mstext.csv",DataFrame)
df3 = CSV.read(flpath*"mmcdh23EC19-20_3CA_mstext.csv",DataFrame)
dfCASite1 = CSV.read(flpath*"mmcdh23EC19-20_CASite1_mstext.csv",DataFrame)
dfCASite2 = CSV.read(flpath*"mmcdh23EC19-20_CASite2_mstext.csv",DataFrame)
dfCASite21 = CSV.read(flpath*"mmcdh23EC19-20_CASite21_mstext.csv",DataFrame)
dfCASite31 = CSV.read(flpath*"mmcdh23EC19-20_CASite31_mstext.csv",DataFrame);
first(df0,3)

In [ ]:
transform!(df0,"Timestep"=>(x->x*2e-6)=>"time (ns)"); select!(df0,Not("Timestep")); select!(df0,ncol(df0),1:ncol(df0)-1);
transform!(df1,"Timestep"=>(x->x*2e-6)=>"time (ns)"); select!(df1,Not("Timestep")); select!(df1,ncol(df1),1:ncol(df1)-1)
transform!(df2,"Timestep"=>(x->x*2e-6)=>"time (ns)"); select!(df2,Not("Timestep")); select!(df2,ncol(df2),1:ncol(df2)-1)
transform!(df3,"Timestep"=>(x->x*2e-6)=>"time (ns)"); select!(df3,Not("Timestep")); select!(df3,ncol(df3),1:ncol(df3)-1)

transform!(dfCASite1,"Timestep"=>(x->x*2e-6)=>"time (ns)"); select!(dfCASite1,Not("Timestep")); select!(dfCASite1,ncol(dfCASite1),1:ncol(dfCASite1)-1)
transform!(dfCASite2,"Timestep"=>(x->x*2e-6)=>"time (ns)"); select!(dfCASite2,Not("Timestep")); select!(dfCASite2,ncol(dfCASite2),1:ncol(dfCASite2)-1)
transform!(dfCASite21,"Timestep"=>(x->x*2e-6)=>"time (ns)"); select!(dfCASite21,Not("Timestep")); select!(dfCASite21,ncol(dfCASite21),1:ncol(dfCASite21)-1)
transform!(dfCASite31,"Timestep"=>(x->x*2e-6)=>"time (ns)"); select!(dfCASite31,Not("Timestep")); select!(dfCASite31,ncol(dfCASite31),1:ncol(dfCASite31)-1)

sort!(df0,"time (ns)"); sort!(df1,"time (ns)"); sort!(df2,"time (ns)"); sort!(df3,"time (ns)");
sort!(dfCASite1,"time (ns)"); sort!(dfCASite2,"time (ns)"); sort!(dfCASite21,"time (ns)"); sort!(dfCASite31,"time (ns)");

first(df0,3)

In [ ]:
p0 = plot(df0[1:100:end,"Delta X(nm)"],-df0[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="0 Ca")
p1 = plot(df1[1:100:end,"Delta X(nm)"],-df1[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="1 Ca")
p2 = plot(df2[1:100:end,"Delta X(nm)"],-df2[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="2 Ca")
p3 = plot(df3[1:100:end,"Delta X(nm)"],-df3[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="3 Ca")

pCASite1 = plot(dfCASite1[1:100:end,"Delta X(nm)"],-dfCASite1[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="CASite1")
pCASite2 = plot(dfCASite2[1:100:end,"Delta X(nm)"],-dfCASite2[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="CASite2")
pCASite21 = plot(dfCASite21[1:100:end,"Delta X(nm)"],-dfCASite21[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="CASite21")
pCASite31 = plot(dfCASite31[1:100:end,"Delta X(nm)"],-dfCASite31[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="CASite31")

lay = @layout [a b c d;e f g h]
plot(p0,p1,p2,p3,pCASite1,pCASite2,pCASite21,pCASite31,layout=lay,size=(300*4,400),margin=6mm)

#### Compute time point at which maximum force occurs and truncate the data

In [ ]:
f0max = maximum(-df0[!,"Left Fx(pN)"]); pos0 = findfirst((-df0[!,"Left Fx(pN)"]).==f0max) 
f1max = maximum(-df1[!,"Left Fx(pN)"]); pos1 = findfirst((-df1[!,"Left Fx(pN)"]).==f1max)
f2max = maximum(-df2[!,"Left Fx(pN)"]); pos2 = findfirst((-df2[!,"Left Fx(pN)"]).==f2max)
f3max = maximum(-df3[!,"Left Fx(pN)"]); pos3 = findfirst((-df3[!,"Left Fx(pN)"]).==f3max)

fCASite1max = maximum(-dfCASite1[!,"Left Fx(pN)"]);   posCASite1 = findfirst((-dfCASite1[!,"Left Fx(pN)"]).==fCASite1max)
fCASite2max = maximum(-dfCASite2[!,"Left Fx(pN)"]);   posCASite2 = findfirst((-dfCASite2[!,"Left Fx(pN)"]).==fCASite2max)
fCASite21max = maximum(-dfCASite21[!,"Left Fx(pN)"]); posCASite21 = findfirst((-dfCASite21[!,"Left Fx(pN)"]).==fCASite21max)
fCASite31max = maximum(-dfCASite31[!,"Left Fx(pN)"]); posCASite31 = findfirst((-dfCASite31[!,"Left Fx(pN)"]).==fCASite31max);

In [ ]:
df0 = df0[1:pos0,:]
df1 = df1[1:pos1,:]
df2 = df2[1:pos2,:]
df3 = df3[1:pos3,:]

dfCASite1 = dfCASite1[1:posCASite1,:]
dfCASite2 = dfCASite2[1:posCASite2,:]
dfCASite21 = dfCASite21[1:posCASite21,:]
dfCASite31 = dfCASite31[1:posCASite31,:];

Plot the truncated portions of force vs extension to verify we've selected the desired parts of the curve.

In [ ]:
p0 = plot(df0[1:100:end,"Delta X(nm)"],-df0[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="0 Ca")
p1 = plot(df1[1:100:end,"Delta X(nm)"],-df1[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="1 Ca")
p2 = plot(df2[1:100:end,"Delta X(nm)"],-df2[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="2 Ca")
p3 = plot(df3[1:100:end,"Delta X(nm)"],-df3[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="3 Ca")

pCASite1 = plot(dfCASite1[1:100:end,"Delta X(nm)"],-dfCASite1[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="CASite1")
pCASite2 = plot(dfCASite2[1:100:end,"Delta X(nm)"],-dfCASite2[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="CASite2")
pCASite21 = plot(dfCASite21[1:100:end,"Delta X(nm)"],-dfCASite21[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="CASite21")
pCASite31 = plot(dfCASite31[1:100:end,"Delta X(nm)"],-dfCASite31[1:100:end,"Left Fx(pN)"],linewidth=2,size=(450,200),ylabel="force (pN)",xlabel="extension (nm)",labels="CASite31")

lay = @layout [a b c d;e f g h]
plot(p0,p1,p2,p3,pCASite1,pCASite2,pCASite21,pCASite31,layout=lay,size=(300*4,400),margin=6mm)

#### Interpolate the force data onto same time axis as the extension data

In [ ]:
ts = [t for t in df0[1,"time (ns)"]:0.001:df0[end,"time (ns)"]]; nts = length(ts)
ys = Vector{Float64}(undef,nts)
for ℓ=1:nts
    tnow = ts[ℓ]
    posa = findlast(df0[!,"time (ns)"].<=tnow);
    if posa == nrow(df0)
        ys[ℓ] = df0[end,"Avg F(pN)"]
        continue
    end
    ta = df0[posa,"time (ns)"]; tb = df0[posa+1,"time (ns)"]
    u = (tnow-ta)/(tb-ta)
    ys[ℓ] = (1-u)*df0[posa,"Avg F(pN)"]+u*df0[posa+1,"Avg F(pN)"]
end;

Mtmp = hcat(reshape(ts,:,1),reshape(ys,:,1))
df0_interp = DataFrame(Mtmp,["time (ns)","Avg F(pN)"]);

In [ ]:
ts = [t for t in df1[1,"time (ns)"]:0.001:df1[end,"time (ns)"]]; nts = length(ts)
ys = Vector{Float64}(undef,nts)
for ℓ=1:nts
    tnow = ts[ℓ]
    posa = findlast(df1[!,"time (ns)"].<=tnow);
    if posa == nrow(df1)
        ys[ℓ] = df1[end,"Avg F(pN)"]
        continue
    end
    ta = df1[posa,"time (ns)"]; tb = df1[posa+1,"time (ns)"]
    u = (tnow-ta)/(tb-ta)
    ys[ℓ] = (1-u)*df1[posa,"Avg F(pN)"]+u*df1[posa+1,"Avg F(pN)"]
end;

Mtmp = hcat(reshape(ts,:,1),reshape(ys,:,1))
df1_interp = DataFrame(Mtmp,["time (ns)","Avg F(pN)"]);

In [ ]:
ts = [t for t in df2[1,"time (ns)"]:0.001:df2[end,"time (ns)"]]; nts = length(ts)
ys = Vector{Float64}(undef,nts)
for ℓ=1:nts
    tnow = ts[ℓ]
    posa = findlast(df2[!,"time (ns)"].<=tnow);
    if posa == nrow(df2)
        ys[ℓ] = df2[end,"Avg F(pN)"]
        continue
    end
    ta = df2[posa,"time (ns)"]; tb = df2[posa+1,"time (ns)"]
    u = (tnow-ta)/(tb-ta)
    ys[ℓ] = (1-u)*df2[posa,"Avg F(pN)"]+u*df2[posa+1,"Avg F(pN)"]
end;

Mtmp = hcat(reshape(ts,:,1),reshape(ys,:,1))
df2_interp = DataFrame(Mtmp,["time (ns)","Avg F(pN)"]);

In [ ]:
ts = [t for t in df3[1,"time (ns)"]:0.001:df3[end,"time (ns)"]]; nts = length(ts)
ys = Vector{Float64}(undef,nts)
for ℓ=1:nts
    tnow = ts[ℓ]
    posa = findlast(df3[!,"time (ns)"].<=tnow);
    if posa == nrow(df3)
        ys[ℓ] = df3[end,"Avg F(pN)"]
        continue
    end
    ta = df3[posa,"time (ns)"]; tb = df3[posa+1,"time (ns)"]
    u = (tnow-ta)/(tb-ta)
    ys[ℓ] = (1-u)*df3[posa,"Avg F(pN)"]+u*df3[posa+1,"Avg F(pN)"]
end;

Mtmp = hcat(reshape(ts,:,1),reshape(ys,:,1))
df3_interp = DataFrame(Mtmp,["time (ns)","Avg F(pN)"]);

In [ ]:
ts = [t for t in dfCASite1[1,"time (ns)"]:0.001:dfCASite1[end,"time (ns)"]]; nts = length(ts)
ys = Vector{Float64}(undef,nts)
for ℓ=1:nts
    tnow = ts[ℓ]
    posa = findlast(dfCASite1[!,"time (ns)"].<=tnow);
    if posa == nrow(dfCASite1)
        ys[ℓ] = dfCASite1[end,"Avg F(pN)"]
        continue
    end
    ta = dfCASite1[posa,"time (ns)"]; tb = dfCASite1[posa+1,"time (ns)"]
    u = (tnow-ta)/(tb-ta)
    ys[ℓ] = (1-u)*dfCASite1[posa,"Avg F(pN)"]+u*dfCASite1[posa+1,"Avg F(pN)"]
end;

Mtmp = hcat(reshape(ts,:,1),reshape(ys,:,1))
dfCASite1_interp = DataFrame(Mtmp,["time (ns)","Avg F(pN)"]);

In [ ]:
ts = [t for t in dfCASite2[1,"time (ns)"]:0.001:dfCASite2[end,"time (ns)"]]; nts = length(ts)
ys = Vector{Float64}(undef,nts)
for ℓ=1:nts
    tnow = ts[ℓ]
    posa = findlast(dfCASite2[!,"time (ns)"].<=tnow);
    if posa == nrow(dfCASite2)
        ys[ℓ] = dfCASite2[end,"Avg F(pN)"]
        continue
    end
    ta = dfCASite2[posa,"time (ns)"]; tb = dfCASite2[posa+1,"time (ns)"]
    u = (tnow-ta)/(tb-ta)
    ys[ℓ] = (1-u)*dfCASite2[posa,"Avg F(pN)"]+u*dfCASite2[posa+1,"Avg F(pN)"]
end;

Mtmp = hcat(reshape(ts,:,1),reshape(ys,:,1))
dfCASite2_interp = DataFrame(Mtmp,["time (ns)","Avg F(pN)"]);

In [ ]:
ts = [t for t in dfCASite21[1,"time (ns)"]:0.001:dfCASite21[end,"time (ns)"]]; nts = length(ts)
ys = Vector{Float64}(undef,nts)
for ℓ=1:nts
    tnow = ts[ℓ]
    posa = findlast(dfCASite21[!,"time (ns)"].<=tnow);
    if posa == nrow(dfCASite21)
        ys[ℓ] = dfCASite21[end,"Avg F(pN)"]
        continue
    end
    ta = dfCASite21[posa,"time (ns)"]; tb = dfCASite21[posa+1,"time (ns)"]
    u = (tnow-ta)/(tb-ta)
    ys[ℓ] = (1-u)*dfCASite21[posa,"Avg F(pN)"]+u*dfCASite21[posa+1,"Avg F(pN)"]
end;

Mtmp = hcat(reshape(ts,:,1),reshape(ys,:,1))
dfCASite21_interp = DataFrame(Mtmp,["time (ns)","Avg F(pN)"]);

In [ ]:
ts = [t for t in dfCASite31[1,"time (ns)"]:0.001:dfCASite31[end,"time (ns)"]]; nts = length(ts)
ys = Vector{Float64}(undef,nts)
for ℓ=1:nts
    tnow = ts[ℓ]
    posa = findlast(dfCASite31[!,"time (ns)"].<=tnow);
    if posa == nrow(dfCASite31)
        ys[ℓ] = dfCASite31[end,"Avg F(pN)"]
        continue
    end
    ta = dfCASite31[posa,"time (ns)"]; tb = dfCASite31[posa+1,"time (ns)"]
    u = (tnow-ta)/(tb-ta)
    ys[ℓ] = (1-u)*dfCASite31[posa,"Avg F(pN)"]+u*dfCASite31[posa+1,"Avg F(pN)"]
end;

Mtmp = hcat(reshape(ts,:,1),reshape(ys,:,1))
dfCASite31_interp = DataFrame(Mtmp,["time (ns)","Avg F(pN)"]);

## Load the DXNDN extension data

In [ ]:
dfext_lkly = CSV.read(flpath*"lkly_mst_DXNDN_ext_v2.csv",DataFrame);
first(dfext_lkly,3)

In [ ]:
dfext_unlkly = CSV.read(flpath*"unlkly_mst_DXNDN_ext.csv",DataFrame);
first(dfext_unlkly,3)

## Combine the extension and force data

In [ ]:
dftmp1 = select(df0_interp,["time (ns)","Avg F(pN)"]); dropmissing!(dftmp1)
dftmp2 = select(dfext_lkly,["time (ns)","0CA Ext (Å)"]); dropmissing!(dftmp2)
dfmst_0 = innerjoin(dftmp1,dftmp2,on="time (ns)"); dropmissing!(dfmst_0)
println("Number rows after innerjoin: $(nrow(dfmst_0))")
first(dfmst_0,3)

In [ ]:
dftmp1 = select(df1_interp,["time (ns)","Avg F(pN)"]); dropmissing!(dftmp1)
dftmp2 = select(dfext_lkly,["time (ns)","1CA Ext (Å)"]); dropmissing!(dftmp2)
dfmst_1 = innerjoin(dftmp1,dftmp2,on="time (ns)"); dropmissing!(dfmst_1)
println("Number rows after innerjoin: $(nrow(dfmst_1))")
first(dfmst_1,3)

In [ ]:
dftmp1 = select(df2_interp,["time (ns)","Avg F(pN)"]); dropmissing!(dftmp1)
dftmp2 = select(dfext_lkly,["time (ns)","2CA Ext (Å)"]); dropmissing!(dftmp2)
dfmst_2 = innerjoin(dftmp1,dftmp2,on="time (ns)"); dropmissing!(dfmst_2)
println("Number rows after innerjoin: $(nrow(dfmst_2))")
first(dfmst_2,3)

In [ ]:
dftmp1 = select(df3_interp,["time (ns)","Avg F(pN)"]); dropmissing!(dftmp1)
dftmp2 = select(dfext_lkly,["time (ns)","3CA Ext (Å)"]); dropmissing!(dftmp2)
dfmst_3 = innerjoin(dftmp1,dftmp2,on="time (ns)"); dropmissing!(dfmst_3)
println("Number rows after innerjoin: $(nrow(dfmst_3))")
first(dfmst_3,3)

In [ ]:
dftmp1 = select(dfCASite1_interp,["time (ns)","Avg F(pN)"]); dropmissing!(dftmp1)
dftmp2 = select(dfext_unlkly,["time (ns)","CASite1 Ext (Å)"]); dropmissing!(dftmp2)
dfmst_CASite1 = innerjoin(dftmp1,dftmp2,on="time (ns)"); dropmissing!(dfmst_CASite1)
println("Number rows after innerjoin: $(nrow(dfmst_CASite1))")
first(dfmst_CASite1,3)

In [ ]:
dftmp1 = select(dfCASite2_interp,["time (ns)","Avg F(pN)"]); dropmissing!(dftmp1)
dftmp2 = select(dfext_unlkly,["time (ns)","CASite2 Ext (Å)"]); dropmissing!(dftmp2)
dfmst_CASite2 = innerjoin(dftmp1,dftmp2,on="time (ns)"); dropmissing!(dfmst_CASite2)
println("Number rows after innerjoin: $(nrow(dfmst_CASite2))")
first(dfmst_CASite2,3)

In [ ]:
dftmp1 = select(dfCASite21_interp,["time (ns)","Avg F(pN)"]); dropmissing!(dftmp1)
dftmp2 = select(dfext_unlkly,["time (ns)","CASite21 Ext (Å)"]); dropmissing!(dftmp2)
dfmst_CASite21 = innerjoin(dftmp1,dftmp2,on="time (ns)"); dropmissing!(dfmst_CASite21)
println("Number rows after innerjoin: $(nrow(dfmst_CASite21))")
first(dfmst_CASite21,3)

In [ ]:
dftmp1 = select(dfCASite31_interp,["time (ns)","Avg F(pN)"]); dropmissing!(dftmp1)
dftmp2 = select(dfext_unlkly,["time (ns)","CASite31 Ext (Å)"]); dropmissing!(dftmp2)
dfmst_CASite31 = innerjoin(dftmp1,dftmp2,on="time (ns)"); dropmissing!(dfmst_CASite31)
println("Number rows after innerjoin: $(nrow(dfmst_CASite31))")
first(dfmst_CASite31,3)

#### Plots of DXNDN extensions vs force

In [ ]:
nstep0 = 1; #nrow(dfmst_0)÷2500
nstep1 = 1; #nrow(dfmst_1)÷2500
nstep2 = 1; #nrow(dfmst_2)÷2500
nstep3 = 1; #nrow(dfmst_3)÷2500

nstepCASite1  = 1; #nrow(dfmst_CASite1)÷2500
nstepCASite2  = 1; #nrow(dfmst_CASite2)÷2500
nstepCASite21 = 1; #nrow(dfmst_CASite21)÷2500
nstepCASite31 = 1; #nrow(dfmst_CASite31)÷2500;

In [ ]:
p0 = scatter(dfmst_0[1:nstep0:end,"0CA Ext (Å)"],dfmst_0[!,"Avg F(pN)"],linewidth=2,labels="0 CA",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)")
p1 = scatter(dfmst_1[1:nstep1:end,"1CA Ext (Å)"],dfmst_1[!,"Avg F(pN)"],linewidth=2,labels="1 CA",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)")
p2 = scatter(dfmst_2[1:nstep2:end,"2CA Ext (Å)"],dfmst_2[!,"Avg F(pN)"],linewidth=2,labels="2 CA",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)")
p3 = scatter(dfmst_3[1:nstep3:end,"3CA Ext (Å)"],dfmst_3[!,"Avg F(pN)"],linewidth=2,labels="3 CA",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)")

pCASite1 = scatter(dfmst_CASite1[1:nstepCASite1:end,"CASite1 Ext (Å)"],dfmst_CASite1[!,"Avg F(pN)"],linewidth=2,labels="CASite1",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)")
pCASite2 = scatter(dfmst_CASite2[1:nstepCASite2:end,"CASite2 Ext (Å)"],dfmst_CASite2[!,"Avg F(pN)"],linewidth=2,labels="CASite2",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)")
pCASite21 = scatter(dfmst_CASite21[1:nstepCASite21:end,"CASite21 Ext (Å)"],dfmst_CASite21[!,"Avg F(pN)"],linewidth=2,labels="CASite21",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)")
pCASite31 = scatter(dfmst_CASite31[1:nstepCASite31:end,"CASite31 Ext (Å)"],dfmst_CASite31[!,"Avg F(pN)"],linewidth=2,labels="CASite31",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)");

In [ ]:
lay = @layout [a b c d;e f g h]
plot(p0,p1,p2,p3,pCASite1,pCASite2,pCASite21,pCASite31,layout=lay,size=(300*4,600),margin=4mm);
savefig("frcvsdxndn.png");

In [ ]:
p0 = histogram2d(dfmst_0[1:nstep0:end,"0CA Ext (Å)"],dfmst_0[1:nstep0:end,"Avg F(pN)"],linewidth=2,labels="0 CA",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)",normalize=:pdf,c=:jet)
p1 = histogram2d(dfmst_1[1:nstep1:end,"1CA Ext (Å)"],dfmst_1[1:nstep1:end,"Avg F(pN)"],linewidth=2,labels="1 CA",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)",normalize=:pdf,c=:jet)
p2 = histogram2d(dfmst_2[1:nstep2:end,"2CA Ext (Å)"],dfmst_2[1:nstep2:end,"Avg F(pN)"],linewidth=2,labels="2 CA",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)",normalize=:pdf,c=:jet)
p3 = histogram2d(dfmst_3[1:nstep3:end,"3CA Ext (Å)"],dfmst_3[1:nstep3:end,"Avg F(pN)"],linewidth=2,labels="3 CA",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)",normalize=:pdf,c=:jet)

pCASite1 = histogram2d(dfmst_CASite1[1:nstepCASite1:end,"CASite1 Ext (Å)"],dfmst_CASite1[1:nstepCASite1:end,"Avg F(pN)"],linewidth=2,labels="CASite1",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)",normalize=:pdf,c=:jet)
pCASite2 = histogram2d(dfmst_CASite2[1:nstepCASite2:end,"CASite2 Ext (Å)"],dfmst_CASite2[1:nstepCASite2:end,"Avg F(pN)"],linewidth=2,labels="CASite2",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)",normalize=:pdf,c=:jet)
pCASite21 = histogram2d(dfmst_CASite21[1:nstepCASite21:end,"CASite21 Ext (Å)"],dfmst_CASite21[1:nstepCASite21:end,"Avg F(pN)"],linewidth=2,labels="CASite21",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)",normalize=:pdf,c=:jet)
pCASite31 = histogram2d(dfmst_CASite31[1:nstepCASite31:end,"CASite31 Ext (Å)"],dfmst_CASite31[1:nstepCASite31:end,"Avg F(pN)"],linewidth=2,labels="CASite31",alpha=0.5,size=(450,200),
             xlabel="extension (Å)",ylabel="force (pN)",normalize=:pdf,c=:jet);

In [ ]:
lay = @layout [a b c d;e f g h]
plot(p0,p1,p2,p3,pCASite1,pCASite2,pCASite21,pCASite31,layout=lay,size=(300*4,600),margin=4mm);
savefig("frcvsdxndn_histogram2d.png");

#### Fit the Gaussian mixture models

In [ ]:
nclusters = 100 # number of Gaussian mixture components
ninit=500 # number of K means iterations to identify component centers
niter=250 #10 # number of expectation maximazation iterations
nfinal=10; #10 # number of iterations in the final step

In [ ]:
M0 = [dfmst_0[i,j] for i=1:nrow(dfmst_0),j=3:-1:2]
gmm0 = GMM(nclusters,M0,method=:kmeans,kind=:diag,nInit=ninit,nIter=niter,nFinal=nfinal)
gmm0_mix = MixtureModel(gmm0);

In [ ]:
dfmst0_xa = minimum(dfmst_0[:,3])
dfmst0_xb = maximum(dfmst_0[:,3])
dfmst0_ya = minimum(dfmst_0[:,2])
dfmst0_yb = maximum(dfmst_0[:,2])
q0 = heatmap(LinRange(dfmst0_xa,dfmst0_xb,251),LinRange(dfmst0_ya,dfmst0_yb,251),(x,y)->pdf(gmm0_mix,[x,y]),
        c=:jet,background_color_inside=plt[1],#clims=(1e-4,0.015),
        #xlims=(7,15),ylims=(0,1250),
        xlabel="extension (Å)",ylabel="force (pN)");

In [ ]:
M1 = [dfmst_1[i,j] for i=1:nrow(dfmst_1),j=3:-1:2]
gmm1 = GMM(nclusters,M1,method=:kmeans,kind=:diag,nInit=ninit,nIter=niter,nFinal=nfinal)
gmm1_mix = MixtureModel(gmm1);

In [ ]:
dfmst1_xa = minimum(dfmst_1[:,3])
dfmst1_xb = maximum(dfmst_1[:,3])
dfmst1_ya = minimum(dfmst_1[:,2])
dfmst1_yb = maximum(dfmst_1[:,2])
q1 = heatmap(LinRange(dfmst1_xa,dfmst1_xb,251),LinRange(dfmst1_ya,dfmst1_yb,251),(x,y)->pdf(gmm1_mix,[x,y]),
        c=:jet,background_color_inside=plt[1],#clims=(1e-4,0.015),
        #xlims=(7,15),ylims=(0,1250),
        xlabel="extension (Å)",ylabel="force (pN)");

In [ ]:
M2 = [dfmst_2[i,j] for i=1:nrow(dfmst_2),j=3:-1:2]
gmm2 = GMM(nclusters,M2,method=:kmeans,kind=:diag,nInit=ninit,nIter=niter,nFinal=nfinal)
gmm2_mix = MixtureModel(gmm2);

In [ ]:
dfmst2_xa = minimum(dfmst_2[:,3])
dfmst2_xb = maximum(dfmst_2[:,3])
dfmst2_ya = minimum(dfmst_2[:,2])
dfmst2_yb = maximum(dfmst_2[:,2])
q2 = heatmap(LinRange(dfmst2_xa,dfmst2_xb,251),LinRange(dfmst2_ya,dfmst2_yb,251),(x,y)->pdf(gmm2_mix,[x,y]),
        c=:jet,background_color_inside=plt[1],#clims=(1e-4,0.015),
        #xlims=(7,15),ylims=(0,1250),
        xlabel="extension (Å)",ylabel="force (pN)");

In [ ]:
M3 = [dfmst_3[i,j] for i=1:nrow(dfmst_3),j=3:-1:2]
gmm3 = GMM(nclusters,M3,method=:kmeans,kind=:diag,nInit=ninit,nIter=niter,nFinal=nfinal)
gmm3_mix = MixtureModel(gmm3);

In [ ]:
dfmst3_xa = minimum(dfmst_3[:,3])
dfmst3_xb = maximum(dfmst_3[:,3])
dfmst3_ya = minimum(dfmst_3[:,2])
dfmst3_yb = maximum(dfmst_3[:,2])
q3 = heatmap(LinRange(dfmst3_xa,dfmst3_xb,251),LinRange(dfmst3_ya,dfmst3_yb,251),(x,y)->pdf(gmm3_mix,[x,y]),
        c=:jet,background_color_inside=plt[1],#clims=(1e-4,0.015),
        #xlims=(7,15),ylims=(0,1250),
        xlabel="extension (Å)",ylabel="force (pN)");

In [ ]:
MCASite1 = [dfmst_CASite1[i,j] for i=1:nrow(dfmst_CASite1),j=3:-1:2]
gmmCASite1 = GMM(nclusters,MCASite1,method=:kmeans,kind=:diag,nInit=ninit,nIter=niter,nFinal=nfinal)
gmmCASite1_mix = MixtureModel(gmmCASite1);

In [ ]:
dfmstCASite1_xa = minimum(dfmst_CASite1[:,3])
dfmstCASite1_xb = maximum(dfmst_CASite1[:,3])
dfmstCASite1_ya = minimum(dfmst_CASite1[:,2])
dfmstCASite1_yb = maximum(dfmst_CASite1[:,2])
qCASite1 = heatmap(LinRange(dfmstCASite1_xa,dfmstCASite1_xb,251),LinRange(dfmstCASite1_ya,dfmstCASite1_yb,251),(x,y)->pdf(gmmCASite1_mix,[x,y]),
        c=:jet,background_color_inside=plt[1],#clims=(1e-4,0.015),
        #xlims=(7,15),ylims=(0,1250),
        xlabel="extension (Å)",ylabel="force (pN)");

In [ ]:
MCASite2 = [dfmst_CASite2[i,j] for i=1:nrow(dfmst_CASite2),j=3:-1:2]
gmmCASite2 = GMM(nclusters,MCASite2,method=:kmeans,kind=:diag,nInit=ninit,nIter=niter,nFinal=nfinal)
gmmCASite2_mix = MixtureModel(gmmCASite2);

In [ ]:
dfmstCASite2_xa = minimum(dfmst_CASite2[:,3])
dfmstCASite2_xb = maximum(dfmst_CASite2[:,3])
dfmstCASite2_ya = minimum(dfmst_CASite2[:,2])
dfmstCASite2_yb = maximum(dfmst_CASite2[:,2])
qCASite2 = heatmap(LinRange(dfmstCASite2_xa,dfmstCASite2_xb,251),LinRange(dfmstCASite2_ya,dfmstCASite2_yb,251),(x,y)->pdf(gmmCASite2_mix,[x,y]),
        c=:jet,background_color_inside=plt[1],#clims=(1e-4,0.015),
        #xlims=(7,15),ylims=(0,1250),
        xlabel="extension (Å)",ylabel="force (pN)");

In [ ]:
MCASite21 = [dfmst_CASite21[i,j] for i=1:nrow(dfmst_CASite21),j=3:-1:2]
gmmCASite21 = GMM(nclusters,MCASite21,method=:kmeans,kind=:diag,nInit=ninit,nIter=niter,nFinal=nfinal)
gmmCASite21_mix = MixtureModel(gmmCASite21);

In [ ]:
dfmstCASite21_xa = minimum(dfmst_CASite21[:,3])
dfmstCASite21_xb = maximum(dfmst_CASite21[:,3])
dfmstCASite21_ya = minimum(dfmst_CASite21[:,2])
dfmstCASite21_yb = maximum(dfmst_CASite21[:,2])
qCASite21 = heatmap(LinRange(dfmstCASite21_xa,dfmstCASite21_xb,251),LinRange(dfmstCASite21_ya,dfmstCASite21_yb,251),(x,y)->pdf(gmmCASite21_mix,[x,y]),
        c=:jet,background_color_inside=plt[1],#clims=(1e-4,0.015),
        #xlims=(7,15),ylims=(0,1250),
        xlabel="extension (Å)",ylabel="force (pN)");

In [ ]:
MCASite31 = [dfmst_CASite31[i,j] for i=1:nrow(dfmst_CASite31),j=3:-1:2]
gmmCASite31 = GMM(nclusters,MCASite31,method=:kmeans,kind=:diag,nInit=ninit,nIter=niter,nFinal=nfinal)
gmmCASite31_mix = MixtureModel(gmmCASite31);

In [ ]:
dfmstCASite31_xa = minimum(dfmst_CASite31[:,3])
dfmstCASite31_xb = maximum(dfmst_CASite31[:,3])
dfmstCASite31_ya = minimum(dfmst_CASite31[:,2])
dfmstCASite31_yb = maximum(dfmst_CASite31[:,2])
qCASite31 = heatmap(LinRange(dfmstCASite31_xa,dfmstCASite31_xb,251),LinRange(dfmstCASite31_ya,dfmstCASite31_yb,251),(x,y)->pdf(gmmCASite31_mix,[x,y]),
        c=:jet,background_color_inside=plt[1],#clims=(1e-4,0.015),
        #xlims=(7,15),ylims=(0,1250),
        xlabel="extension (Å)",ylabel="force (pN)");

In [ ]:
lay = @layout [a b c d;e f g h]
pmst = plot(p0,p1,p2,p3,pCASite1,pCASite2,pCASite21,pCASite31,layout=lay,size=(600*4,600),margin=12mm);
savefig("frcvsdxndn_histogram2d.png");

In [ ]:
lay = @layout [a b c d;e f g h]
qmst = plot(q0,q1,q2,q3,qCASite1,qCASite2,qCASite21,qCASite31,layout=lay,size=(600*4,600),margin=12mm)
savefig("frcvsdxndn_gmm.png");

In [ ]:
println("EC19-20 data:")
plot(pmst)

In [ ]:
println("Gaussian mixture model:")
plot(qmst)

In [ ]:
lay = @layout [a1 a2 a3 a4;b1 b2 b3 b4;c1 c2 c3 c4;d1 d2 d3 d4]
pqmst = plot(p0,q0,p1,q1,p2,q2,p3,q3,pCASite1,qCASite1,pCASite2,qCASite2,pCASite21,qCASite21,pCASite31,qCASite31,
             layout=lay,size=(600*4,600*2),margin=12mm)

In [ ]:
savefig("frcvsdxndn_histgmm.pdf")
savefig("frcvsdxndn_histgmm.png");

In [ ]:
lay = @layout [a1 a2;b1 b2;c1 c2;d1 d2]
plot(p0,q0,p1,q1,p2,q2,p3,q3,layout=lay,size=(600*2,600*2),margin=12mm)

In [ ]:
savefig("frcvsdxndn_histgmm_typicals.pdf")
savefig("frcvsdxndn_histgmm_typicals.png");

In [ ]:
lay = @layout [a1 a2;b1 b2;c1 c2;d1 d2]
plot(pCASite1,qCASite1,pCASite2,qCASite2,pCASite21,qCASite21,pCASite31,qCASite31,layout=lay,size=(600*2,600*2),margin=12mm)

In [ ]:
savefig("frcvsdxndn_histgmm_atypicals.pdf")
savefig("frcvsdxndn_histgmm_atypicals.png");

## Export the mixture models
The exported `csv` file is recorded as
- Each column is a different Ca2+ configuration at a linker,
- The first 25 rows are the mixture weights for the different Gaussian components,
- The 50 rows after that are the component means organized so that consecutive blocks of two vectors are the 2d components of one mean,
- The last 50 rows after that are the component variances organized so that consective blocks of two vectors are the 2d components of one diagnoal variance.

In the `Ca2+ Site Probabilities.ipynb` the ordering of $\mathrm{Ca}^{2+}$ states was p010 p100 p101 p110 p000 p001 p011 p111.

In [ ]:
Mexp = Matrix{Float64}(undef,5nclusters,8)
pos = 0
for gm in [gmmCASite2,gmmCASite1,gmmCASite31,gmmCASite21,gmm0,gmm1,gmm2,gmm3]
    pos += 1
    Mexp[1:nclusters,pos]   .= gm.w[:]
    Mexp[(nclusters+1):(3nclusters),pos]  .= transpose(gm.μ)[:]
    Mexp[(3nclusters+1):(5nclusters),pos] .= transpose(gm.Σ)[:]
end
dfexp = DataFrame(Mexp,["CASite2","CASite1","CASite31","CASite21","0 Ca","1 Ca","2 Ca","3 Ca"]);

In [ ]:
CSV.write("gmm_extvsfrc.csv",dfexp);

## Hyperparameter section for number of GMM clusters
This is an optional selection for performing cross-validation selection of the number of GMM clusters to use.

In [ ]:
rgncomps = [5,10,25,50,75,100,125,150];

In [ ]:
"""
    rungmmcv(X;nfolds=5,rgncomps=rgncomps,niter=niter,minvar=minvar)
Run cross-validation of the GMM on data X to identify the number of clusters.
The routine returns a matrix of shape nhypers x nfolds whose components are the log-likelihood of the fitted model on the withheld fold. It also returns the indices for the folds.
- X is the data set, arranged as nsmp x 2 matrix
- nfolds is the number of cross validation folds
- rgncomps is the range values for the number of clusters
"""
function rungmmcv(X;
                  nfolds=5,rgncomps=rgncomps,niter=niter,ninit=ninit,nfinal=nfinal)
    nX,d = size(X)
    foldsz = nX÷nfolds; nXf = foldsz*nfolds
    
    # split data into folds by grabbing row indices of X
    σ = randperm(nX)
    Σ = [σ[(j-1)*foldsz+i] for i=1:foldsz,j=1:nfolds]
    
    # Run CV over the folds
    nhypers = length(rgncomps)
    lglk = Matrix{Float64}(undef,nhypers,nfolds)
    for ℓi=1:nhypers
        for ℓj=1:nfolds
            # grab training data
            ids = vcat([i for i in 1:(ℓj-1)],[i for i in (ℓj+1):nfolds])
            Xtrain = vcat([X[Σ[:,j],:] for j in ids]...)
            
            # Plug in
            gmm = GMM(rgncomps[ℓi],Xtrain,method=:kmeans,kind=:diag,nInit=ninit,nIter=niter,nFinal=nfinal)
            gmm_mix = MixtureModel(gmm)
            
            # grab test data
            Xtest = X[Σ[:,ℓj],:]
            
            # compute log-likelihood loss over test set
            lglk[ℓi,ℓj] = 0.
            for ℓk=1:foldsz
                xtmp = @SVector [Xtest[ℓk,1],Xtest[ℓk,2]]
                lglk[ℓi,ℓj] += logpdf(gmm_mix,xtmp)
            end
        end
    end
    
    d = Dict{Symbol,Any}(:loss=>lglk,:folds=>Σ)
    return d
end;

In [ ]:
d0 = rungmmcv(M0);

In [ ]:
d1 = rungmmcv(M1);

In [ ]:
d2 = rungmmcv(M2);

In [ ]:
d3 = rungmmcv(M3);

In [ ]:
dCASite1 = rungmmcv(MCASite1);

In [ ]:
dCASite2 = rungmmcv(MCASite2);

In [ ]:
dCASite21 = rungmmcv(MCASite21);

In [ ]:
dCASite31 = rungmmcv(MCASite31);

#### Postprocess the fold log-likelihood data and find best fitting values

In [ ]:
hypers = [r for r in rgncomps]
CVmst = hcat([d[:loss] for d in [d0,d1,d2,d3,dCASite1,dCASite2,dCASite21,dCASite31]]...)
CVmst = sum(CVmst,dims=2)
println("Best # of components was $(hypers[findfirst((CVmst.==maximum(CVmst))[:])])")
dfcv = DataFrame(hcat(hypers,CVmst),["hyper","Σ log-lk"])